### Python Request Utility using `requests` Library and Builder Pattern

In [25]:
import requests
import logging
from typing import Dict, Any, Optional
from requests.adapters import HTTPAdapter, Retry


# --- Configure Logging ---
logger = logging.getLogger("RequestUtil")
logger.setLevel(logging.DEBUG)

if not logger.handlers:   # 👈 prevents duplicate handlers
    console_handler = logging.StreamHandler()
    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(message)s"
    )
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)


class RequestBuilder:
    def __init__(self):
        self._method: str = "GET"
        self._url: Optional[str] = None
        self._headers: Dict[str, str] = {}
        self._params: Dict[str, Any] = {}
        self._data: Dict[str, Any] = {}
        self._json: Dict[str, Any] = {}
        self._timeout: int = 30
        self._proxies: Dict[str, str] = {}
        self._max_retries: int = 3
        self._backoff_factor: float = 0.3
        self._status_forcelist = (500, 502, 503, 504)

    # --- Builder methods ---
    def set_method(self, method: str) -> "RequestBuilder":
        self._method = method
        return self
    
    def set_url(self, url: str) -> "RequestBuilder":
        self._url = url
        return self
    
    def set_headers(self, headers: Dict[str, str]) -> "RequestBuilder":
        self._headers = headers
        return self
    
    def set_params(self, params: Dict[str, Any]) -> "RequestBuilder":
        self._params = params
        return self
    
    def set_data(self, data: Dict[str, Any]) -> "RequestBuilder":
        self._data = data
        return self
    
    def set_json(self, json: Dict[str, Any]) -> "RequestBuilder":
        self._json = json
        return self
    
    def set_timeout(self, timeout: int) -> "RequestBuilder":
        self._timeout = timeout
        return self
    
    def set_proxy(self, proxies: Dict[str, str]) -> "RequestBuilder":
        """
        Example: {"http": "http://localhost:8080", "https": "http://localhost:8080"}
        """
        self._proxies = proxies
        return self
    
    def set_retries(
            self,
            max_retries: int = 3,
            backoff_factor: float = 0.3,
            status_forcelist = (500, 502, 503, 504)
    ) -> "RequestBuilder":
        self._max_retries = max_retries
        self._backoff_factor = backoff_factor
        self._status_forcelist = status_forcelist
        return self
    
    # --- Execute method ---
    def execute(self) -> requests.Response:
        if not self._url:
            raise ValueError("URL must be set before executing the request")
        
        session = requests.Session()

        # Setup Retry Strategy
        retries = Retry(
            total=self._max_retries,
            backoff_factor=self._backoff_factor,
            status_forcelist=self._status_forcelist,
            allowed_methods=["HEAD", "GET", "OPTIONS", "POST", "PUT", "DELETE"]
        )

        adapter = HTTPAdapter(max_retries=retries)
        session.mount("http://", adapter)
        session.mount("https://", adapter)

        try:
            logger.debug(
                f"Executing {self._method} request: "
                f"url={self._url}, headers={self._headers}, "
                f"params={self._params}, data={self._data}, "
                f"json={self._json}, timeout={self._timeout}, "
                f"proxies={self._proxies}"
            )
            response = session.request(
                method=self._method,
                url=self._url,
                headers=self._headers,
                params=self._params,
                data=self._data if self._data else None,
                json=self._json if self._json else None,
                timeout=self._timeout,
                proxies=self._proxies if self._proxies else None
            )
            logger.info(
                f"Response received: status={response.status_code}, "
                f"content_length={len(response.content)}"
            )
            return response
        except requests.RequestException as e:
            logger.error(f"Request failed: {e}")
            raise

# --- Convenience API for common methods ---
class RequestUtil:
    @staticmethod
    def get(url: str) -> RequestBuilder:
        return RequestBuilder().set_method("GET").set_url(url)
    
    @staticmethod
    def post(url: str) -> RequestBuilder:
        return RequestBuilder().set_method("POST").set_url(url)
    
    @staticmethod
    def put(url: str) -> RequestBuilder:
        return RequestBuilder().set_method("PUT").set_url(url)
    
    @staticmethod
    def delete(url: str) -> RequestBuilder:
        return RequestBuilder().set_method("DELETE").set_url(url)
    


if __name__ == "__main__":
    # Example GET with retries
    response = (
        RequestUtil.get("https://jsonplaceholder.typicode.com/posts")
        .set_params({"userId": 1})
        .set_headers({"Accept": "application/json"})
        .set_timeout(10)
        .set_retries(max_retries=3, backoff_factor=1)
        .execute()
    )
    print(response.json())


2025-09-07 22:08:04,830 - DEBUG - Executing GET request: url=https://jsonplaceholder.typicode.com/posts, headers={'Accept': 'application/json'}, params={'userId': 1}, data={}, json={}, timeout=10, proxies={}
2025-09-07 22:08:05,833 - INFO - Response received: status=200, content_length=2726


[{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}, {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'}, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}, {'userId': 1, 'id': 4, 'title': 'eum et est occaecati', 'body': 'ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rerum culpa\nquis hic c